# Importing libraries

No algorithm consistently outperformed all others in either differential survival or enriched clinical parameters. With respect to survival, `MCCA` had the best prognostic value, while `MultiNMF` was second and `LRACluster` third.

Since my system currently fails to install `PMA` (`MCCA`) successfully, I install `SNFtool` (`SNF`) instead.

In [1]:
# LRACluster
library('LRAcluster')

# MultiNMF
library('NMF')

# SNF
library('SNFtool')

Loading required package: pkgmaker

Loading required package: registry

Loading required package: rngtools

Loading required package: cluster

NMF - BioConductor layer [OK] | Shared memory capabilities [NO: bigmemory] | Cores 9/10

  To enable shared memory capabilities, try: install.extras('
NMF
')



In [2]:
ALGORITHM.NAMES = c('lracluster', 'nmf', 'snf')
ALGORITHM.DISPLAY.NAMES = as.list(c('LRAcluster', 'MultiNMF', 'SNF'))
names(ALGORITHM.DISPLAY.NAMES) = ALGORITHM.NAMES

# Loading data

In task 1, I chose 3 multi-omics cancer datasets: `BIC` (dataset of breast), `LIHC` (dataset of the liver) and `SKCM` (dataset of melanoma).

In [3]:
SUBTYPES.DATA = list(
    list(name='breast', only.primary=T, is.rna.seq=T, is.mirna.seq=T, display.name='BIC'),
    list(name='liver', only.primary=T, is.rna.seq=T, is.mirna.seq=T, display.name='LIHC'),
    list(name='melanoma', only.primary=F, is.rna.seq=T, is.mirna.seq=T, display.name='SKCM'))

In [4]:
get.dataset.dir.path <- function() {
    return('./data/cancer/')
}

# Importing functions

In [5]:
get.raw.data <- function(subtype.name,
                         datasets.path = get.dataset.dir.path(),
                         only.primary=NA) {
    omics.dir = file.path(datasets.path, subtype.name)
    omics.files = list.files(omics.dir)
    omics.files = setdiff(omics.files, c('survival'))  
    raw.data = lapply(file.path(omics.dir, omics.files), read.table)
    
    if (!is.na(only.primary)) {
        raw.data = lapply(raw.data, function(x) filter.non.tumor.samples(x, only.primary = only.primary))
    }
    name.corrected.data = fix.patient.names(raw.data)
    patients.intersection = Reduce(intersect, lapply(name.corrected.data, colnames))
    ret.data = lapply(name.corrected.data, function(datum) datum[,patients.intersection])  
    return(ret.data)
}

In [6]:
filter.non.tumor.samples <- function(raw.datum, only.primary=only.primary) {
    # 01 is primary, 06 is metastatic, 03 is blood derived cancer
    if (!only.primary)
        return(raw.datum[,substring(colnames(raw.datum), 14, 15) %in% c('01', '03', '06')])
    else
        return(raw.datum[,substring(colnames(raw.datum), 14, 15) %in% c('01')])
}

In [7]:
get.fixed.names <- function(patient.names, include.type=F) {
    # fix the TCGA names to only include the patient ids
    if (include.type) {
        return(gsub('-', '\\.', toupper(substring(patient.names, 1, 15))))
    } else {
        return(gsub('-', '\\.', toupper(substring(patient.names, 1, 12))))  
    }
}

fix.patient.names <- function(subtype.raw.data, include.type=F) {
    for (i in 1:length(subtype.raw.data)) {
        colnames(subtype.raw.data[[i]]) = get.fixed.names(colnames(subtype.raw.data[[i]]),
                                                          include.type)
    }
    return(subtype.raw.data)
}

In [8]:
set.omics.list.attr <- function(subtype.raw.data, subtype.data) {
    attr(subtype.raw.data[[1]], 'is.seq') = subtype.data$is.rna.seq
    attr(subtype.raw.data[[2]], 'is.seq') = F
    attr(subtype.raw.data[[3]], 'is.seq') = subtype.data$is.mirna.seq
    return(subtype.raw.data)
}

In [9]:
log.and.normalize <- function(omics.data, subtype.data, normalize=T,
                                filter.var=F) {
    # filter features with no variance at all
    for (i in 1:length(omics.data)) {
        omics.data[[i]] = omics.data[[i]][apply(omics.data[[i]], 1, var) > 0,]
    }
                  
    for (i in 1:length(omics.data)) {
        if ('is.seq' %in% attributes(omics.data[[i]])$names) {
            omics.data[[i]] = log(1+omics.data[[i]])
        }
    }
    
    if (filter.var) {
        omics.data = lapply(omics.data, keep.high.var.features)
    }
    
    if (normalize) {
        omics.data = lapply(omics.data, normalize.matrix)    
    }
    
    return(omics.data)
}

In [10]:
normalize.matrix <- function(data.matrix) {
    temp = data.matrix - rowMeans(data.matrix)
    should.keep = (apply(temp, 1, sd) != 0)
    return ((temp / apply(temp, 1, sd))[should.keep, ])
}

keep.high.var.features <- function(omic, num.features=2000) {
    if (nrow(omic) < num.features) {
        return(omic)
    } else {
        feature.vars = apply(omic, 1, var)
        threshold = feature.vars[order(feature.vars, decreasing = T)][num.features]
        return(omic[feature.vars >= threshold,])    
    }
}

In [11]:
run.lracluster <- function(omics.list, subtype.data) {
    omics.list = log.and.normalize(omics.list, subtype.data, normalize = F)
    
    subtype = subtype.data$name
    start = Sys.time()
    
    dim.range = 1:MAX.NUM.CLUSTERS
    all.clustering.results = list()
    
    omics.matrix.list = lapply(omics.list, as.matrix)
    for (dimension in dim.range) {
        print(paste('running lra cluster for dimension', dimension))
        data.names = c('gene expression', 'methylation', 'miRNA expression')
        clustering.results = LRAcluster(omics.matrix.list, 
                                        rep('gaussian', length(omics.list)), 
                                        dimension=dimension, data.names)
        all.clustering.results[[dimension]] = clustering.results
    }
    explained.var = sapply(all.clustering.results, function(x) x$potential)
    print(explained.var)
    dimension = get.elbow(explained.var, is.max=F)
    print(dimension)
    solution = all.clustering.results[[dimension]]$coordinate
    
    sils = c()
    clustering.per.num.clusters = list()
    for (num.clusters in 2:MAX.NUM.CLUSTERS) {
        print(paste('running kmeans in lra cluster for num clusters', num.clusters))
        cur.clustering = kmeans(t(solution), num.clusters, iter.max=100, nstart=60)$cluster
        sil = get.clustering.silhouette(list(solution), cur.clustering)
        sils = c(sils, sil)
        clustering.per.num.clusters[[num.clusters - 1]] = cur.clustering
    }
    print(sils)
    # NOTE: the next line contains an error. We mistakenly selected the minimal rather maximal silhouette.
    # See more details in: http://acgt.cs.tau.ac.il/multi_omic_benchmark/download.html.
    chosen.clustering = clustering.per.num.clusters[[which.min(sils)]]
    time.taken = as.numeric(Sys.time() - start, units='secs')
    return(list(clustering=chosen.clustering, timing=time.taken))
}

In [12]:
run.snf <- function(omics.list, subtype.data) {
    start = Sys.time()
    omics.list = log.and.normalize(omics.list, subtype.data)
    subtype = subtype.data$name
    alpha=0.5
    T.val=30
    num.neighbors = round(ncol(omics.list[[1]]) / 10)
    similarity.data = lapply(omics.list, function(x) {affinityMatrix(dist2(as.matrix(t(x)),as.matrix(t(x))), 
                                                                     num.neighbors, alpha)})
    if (length(similarity.data) == 1) {
        W = similarity.data[[1]]
    } else {
        W = SNF(similarity.data, num.neighbors, T.val)  
    }
    
    num.clusters = estimateNumberOfClustersGivenGraph(W, 2:MAX.NUM.CLUSTERS)[[3]]  
    clustering = spectralClustering(W, num.clusters)
    time.taken = as.numeric(Sys.time() - start, units='secs')
    return(list(clustering=clustering, timing=time.taken))
}

In [13]:
run.snf <- function(omics.list, subtype.data) {
    start = Sys.time()
    omics.list = log.and.normalize(omics.list, subtype.data)
    subtype = subtype.data$name
    alpha=0.5
    T.val=30
    num.neighbors = round(ncol(omics.list[[1]]) / 10)
    similarity.data = lapply(omics.list, function(x) {affinityMatrix(dist2(as.matrix(t(x)),as.matrix(t(x))), 
                                                                     num.neighbors, alpha)})
    if (length(similarity.data) == 1) {
        W = similarity.data[[1]]
    } else {
        W = SNF(similarity.data, num.neighbors, T.val)  
    }
    
    num.clusters = estimateNumberOfClustersGivenGraph(W, 2:MAX.NUM.CLUSTERS)[[3]]  
    clustering = spectralClustering(W, num.clusters)
    time.taken = as.numeric(Sys.time() - start, units='secs')
    return(list(clustering=clustering, timing=time.taken))
}

# Saving results

In [14]:
MAX.NUM.CLUSTERS = 15

In [15]:
get.clustering.results.dir.path <- function() {
    return('results_task_1_b')
}

In [16]:
run.benchmark <- function() {
    for (i in 1:length(SUBTYPES.DATA)) {
        current.subtype.data = SUBTYPES.DATA[[i]]
        subtype = current.subtype.data$name
        subtype.raw.data = get.raw.data(subtype, 
                                        only.primary=current.subtype.data$only.primary)
        
        subtype.raw.data = set.omics.list.attr(subtype.raw.data, 
                                               current.subtype.data)
        
        for (algorithm.name in ALGORITHM.NAMES) {
            for (j in c('all', '1', '2', '3')) {
                set.seed(42)
                print(paste('subtype', subtype, 'running algorithm', algorithm.name, j))
                clustering.path = file.path(get.clustering.results.dir.path(),
                                            paste(subtype, algorithm.name, j, sep='_'))
                timing.path = file.path(get.clustering.results.dir.path(),
                                        paste(subtype, algorithm.name, j, 'timing', sep='_'))
    
    
                if (!file.exists(clustering.path)) {
                    algorithm.func.name = paste0('run.', algorithm.name)
                    algorithm.func = get(algorithm.func.name)
            if (j== 'all') {
            cur.iteration.data = subtype.raw.data
        } else {
            cur.iteration.data = subtype.raw.data[as.numeric(j)]
        }
                algorithm.ret = algorithm.func(cur.iteration.data, current.subtype.data)
                clustering = algorithm.ret$clustering
                timing = algorithm.ret$timing
                print('before saving')
                save(clustering, file = clustering.path)
                save(timing, file = timing.path)
                }
            }
        }
    }
}

In [ ]:
run.benchmark()